In [ ]:
import pandas as pd
import numpy as np
import os
from gensim.models.word2vec import Word2Vec
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import
nltk.download('wordnet')

# os.chdir('/content/drive/My Drive/OEM/Final datasets')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
scenario2_df = pd.read_csv('model2inputs.csv')
scenario2_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace=True, errors='ignore')
scenario2_df.head()

,mdr_text.mdr_text_key,mdr_text.text_type_code,mdr_text.text,device.manufacturer_d_state,device.lot_number,device.manufacturer_d_postal_code,device.model_number,device.date_received,device.device_report_product_code,device.device_availability,device.generic_name,device.manufacturer_d_name,device.manufacturer_d_country,device.brand_name,device.device_age_text,device.catalog_number,device.date_returned_to_manufacturer,device.expiration_date_of_device,event_type,report_number,type_of_report,product_problem_flag,date_received,reprocessed_and_reused_flag,previous_use_code,mdr_report_key,report_source_code,device_date_of_manufacturer,single_use_flag,remedial_action,distributor_name,product_problems,manufacturer_link_flag,report_to_manufacturer,date_report_to_manufacturer,device.openfda.device_name,device.openfda.fei_number,device.openfda.device_class,device.openfda.medical_specialty_description,op_flag
0,8094441,Additional Manufacturer Narrative,THIS IS AN INITIAL REPORT. AN INVESTIGATION IS...,CA,NaN,95054,NaN,20080220,GKZ,Yes,AUTOMATED HEMATOLOGY ANALYZER,ABBOTT DIAGNOSTICS DIVISION/CELLDYN,US,CELL-DYN SAPPHIRE ANALYZER,DA,8H00-01,NaN,NaN,Malfunction,2919069-2008-00381,"['Initial submission', 'Followup']",Y,20080220,N,U,1000000,Manufacturer report,NaN,N,['Replace'],NaN,"['Syringe, defective', 'Device Displays Incorr...",Y,NaN,NaN,"Counter, Differential Cell",['2919069'],2,Hematology,1
1,8303917,Additional Manufacturer Narrative,(B)(4). CONCOMITANT MEDICAL PRODUCTS: CELL-DYN...,CA,NaN,95054,NaN,20080220,GKZ,Yes,AUTOMATED HEMATOLOGY ANALYZER,ABBOTT DIAGNOSTICS DIVISION/CELLDYN,US,CELL-DYN SAPPHIRE ANALYZER,DA,8H00-01,NaN,NaN,Malfunction,2919069-2008-00381,"['Initial submission', 'Followup']",Y,20080220,N,U,1000000,Manufacturer report,NaN,N,['Replace'],NaN,"['Syringe, defective', 'Device Displays Incorr...",Y,NaN,NaN,"Counter, Differential Cell",['2919069'],2,Hematology,1
2,788224,Description of Event or Problem,THE CUSTOMER CONTACTED ABBOTT REGARDING HEMOGL...,CA,NaN,95054,NaN,20080220,GKZ,Yes,AUTOMATED HEMATOLOGY ANALYZER,ABBOTT DIAGNOSTICS DIVISION/CELLDYN,US,CELL-DYN SAPPHIRE ANALYZER,DA,8H00-01,NaN,NaN,Malfunction,2919069-2008-00381,"['Initial submission', 'Followup']",Y,20080220,N,U,1000000,Manufacturer report,NaN,N,['Replace'],NaN,"['Syringe, defective', 'Device Displays Incorr...",Y,NaN,NaN,"Counter, Differential Cell",['2919069'],2,Hematology,1
3,189010215,Additional Manufacturer Narrative,THE INSULIN PUMP WAS RECEIVED WITH A CRACKED C...,NaN,HG2UREC,00777-3869,MMT-1715K,20200427,OZO,Device was returned to manufacturer,"ARTIFICIAL PANCREAS DEVICE SYSTEM, THRESHOLD S...",MEDTRONIC PUERTO RICO OPERATIONS CO.,*,630G INSULIN PUMP MMT-1715K 630G BLACK MG,NaN,MMT-1715K,20200319.0,NaN,Malfunction,2032227-2020-110169,['Initial submission'],Y,20200427,N,U,10000000,Manufacturer report,20180927.0,N,[''],NaN,"['No Apparent Adverse Event', None]",Y,NaN,NaN,"Automated Insulin Dosing , Threshold Suspend",['3003166194'],3,Unknown,0
4,189010216,Description of Event or Problem,INFORMATION RECEIVED BY MEDTRONIC INDICATED TH...,NaN,HG2UREC,00777-3869,MMT-1715K,20200427,OZO,Device was returned to manufacturer,"ARTIFICIAL PANCREAS DEVICE SYSTEM, THRESHOLD S...",MEDTRONIC PUERTO RICO OPERATIONS CO.,*,630G INSULIN PUMP MMT-1715K 630G BLACK MG,NaN,MMT-1715K,20200319.0,NaN,Malfunction,2032227-2020-110169,['Initial submission'],Y,20200427,N,U,10000000,Manufacturer report,20180927.0,N,[''],NaN,"['No Apparent Adverse Event', None]",Y,NaN,NaN,"Automated Insulin Dosing , Threshold Suspend",['3003166194'],3,Unknown,0


In [ ]:
scenario2_df.isna().sum()

mdr_text.mdr_text_key                               0
mdr_text.text_type_code                             0
mdr_text.text                                       6
device.manufacturer_d_state                      4690
device.lot_number                                7523
device.manufacturer_d_postal_code                2076
device.model_number                              3130
device.date_received                                0
device.device_report_product_code                   0
device.device_availability                         16
device.generic_name                               234
device.manufacturer_d_name                         15
device.manufacturer_d_country                    2033
device.brand_name                                  28
device.device_age_text                          10136
device.catalog_number                            1128
device.date_returned_to_manufacturer            10884
device.expiration_date_of_device                15655
event_type                  

In [ ]:
recomm_df['op_flag'] = recomm_df['device_date_returned_to_manufacturer'].notna().astype(int)

In [ ]:
scenario2_input_data = scenario2_input_data[['device_device_report_product_code', 'device_brand_name', 'distributor_name', 'device_manufacturer_d_name', 'device_catalog_number', 'device_openfda.medical_specialty_description', 'device_model_number', 'device_generic_name', 'op_flag']]

KeyError: ignored

In [ ]:
recomm_df.head()

In [ ]:
recomm_df.shape

In [ ]:
recomm_df.to_csv('recomm_sys_updated.csv')

In [ ]:
key_device_features = recomm_df['KEY_DEVICE_DATA'].tolist()
key_device_features

In [ ]:
lemmatizer = WordNetLemmatizer()
cleaned_features = []
for doc in key_device_features:
    words = regexp_tokenize(doc.lower(), r'[A-Za-z0-9]+')
    words = [lemmatizer.lemmatize(w) for w in words]
    cleaned_features.append(words)
cleaned_features

In [ ]:
len(cleaned_features)

In [ ]:
' '.join(cleaned_features[0])

In [ ]:
recomm_df['CLEANED_FEATURES'] = [' '.join(text) for text in cleaned_features]
recomm_df.head()

In [ ]:
recomm_df.isna().sum()

In [ ]:
from gensim.models import doc2vec

In [ ]:
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield doc2vec.TaggedDocument(list_of_words, [i])

In [ ]:
train_data = list(create_tagged_document(cleaned_features))
print(train_data[:1])

In [ ]:
model = doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_data)

In [ ]:
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save('scenario2_recom_model.pkl')

In [ ]:
sentence = 'ventilator'
words = sentence.split()
print(model.infer_vector(words))

In [ ]:
new_vector = model.infer_vector(words)

In [ ]:
sims = model.docvecs.most_similar([new_vector])

In [ ]:
sims

In [ ]:
most_similar_docs = []
for d in sims:
    most_similar_docs.append(cleaned_features[d[0]])

In [ ]:
most_similar_docs

In [ ]:
predicted_docs = []
for text in most_similar_docs:
    text = ' '.join([w for w in text])
    predicted_docs.append(text)

In [ ]:
predicted_docs[0]

In [ ]:
Index_label = recomm_df[recomm_df['CLEANED_FEATURES'] == predicted_docs[0]].index.tolist()

In [ ]:
Index_label[:5]

In [ ]:
recomm_df.loc[152450,'KEY_DEVICE_DATA']